In [1]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook, tqdm
from collections import deque

In [2]:
class AI_Trader():
  
    def __init__(self, state_size, action_space = 3, model_name = "AITrader"):
        self.state_size = state_size
        self.action_space = action_space
        self.memory = deque(maxlen = 2000)
        self.model_name = model_name

        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_final = 0.01
        self.epsilon_decay = 0.995
        self.model = self.model_builder()
    
    def model_builder(self):
        model = tf.keras.models.Sequential()
        model.add(tf.keras.layers.Dense(units = 32, activation = "relu", input_dim = self.state_size))
        model.add(tf.keras.layers.Dense(units = 64, activation = "relu"))
        model.add(tf.keras.layers.Dense(units = 128, activation = "relu"))
        model.add(tf.keras.layers.Dense(units = self.action_space, activation = "linear"))
        model.compile(loss = "mse", optimizer = tf.keras.optimizers.Adam(lr = 0.001))
        return model
  
    def trade(self, state):
        if random.random() <= self.epsilon:
            return random.randrange(self.action_space)

        actions = self.model.predict(state)
        return np.argmax(actions[0])
  
    def batch_train(self, batch_size):
        batch = []
        for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
            batch.append(self.memory[i])

        for state, action, reward, next_state, done in batch:
            if not done:
                reward = reward + self.gamma * np.max(self.model.predict(next_state))

        target = self.model.predict(state)
        target[0][action] = reward

        self.model.fit(state, target, epochs=1, verbose=0)

        if self.epsilon > self.epsilon_final:
            self.epsilon *= self.epsilon_decay

    def save(self, path):
        self.model.save(path)

In [3]:
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [4]:
def stocks_price_format(n):
    if n < 0:
        return "- $ {0:2f}".format(abs(n))
    else:
        return "$ {0:2f}".format(abs(n))

In [5]:
def dataset_loader(file_path):
    dataset = pd.read_csv(file_path)
    start_date = str(dataset.index[0]).split()[0]
    end_date = str(dataset.index[-1]).split()[0]
    value = dataset['valor']
    return value

In [6]:
def state_creator(data, timestep, window_size):
    starting_id = timestep - window_size + 1
  
    if starting_id >= 0:
        windowed_data = data[starting_id:timestep + 1]
    else:
        windowed_data = - starting_id * [data[0]] + list(data[0:timestep + 1])
        
    state = []
    for i in range(window_size - 1):
        state.append(sigmoid(list(windowed_data)[i + 1] - list(windowed_data)[i]))
    
    return np.array([state])

In [7]:
data = dataset_loader('/home/henrique/Documentos/dolar3.csv')

### Hyper parâmetros

In [8]:
window_size = 10
episodes = 100
batch_size = 32
data_samples = len(data) - 1

In [9]:
trader = AI_Trader(window_size)

In [10]:
trader.model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                352       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 387       
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________


In [11]:
initial_amount = 15

for episode in range(1, episodes + 1):
    print("Episode: {}/{}".format(episode, episodes))
    state = state_creator(data, 0, window_size + 1)
    total_profit = 0
    trader.inventory = []
    operation_amount = initial_amount
    for t in tqdm(range(data_samples)):
        action = trader.trade(state)
        next_state = state_creator(data, t + 1, window_size + 1)
        reward = 0

        if action == 1 and operation_amount > 0: # Comprando uma ação
            trader.inventory.append(data[t])
            operation_amount -= 1
            print("AI Trader bought: ", stocks_price_format(data[t]))
        elif action == 2 and len(trader.inventory) > 0: # Vendendo uma ação  
            buy_price = trader.inventory.pop(0)
            reward = max(data[t] - buy_price, 0)
            total_profit += data[t] - buy_price
            operation_amount += 1
            print("AI Trader sold: ", stocks_price_format(data[t]), " Profit: " + stocks_price_format(data[t] - buy_price))

        if t == data_samples - 1:
            done = True
        else:
            done = False

        trader.memory.append((state, action, reward, next_state, done))

        state = next_state

        if done:
            print("########################")
            print("Total profit: {}".format(total_profit))
            print("Total operation: {}".format(operation_amount))
            print("########################")

        if len(trader.memory) > batch_size and len(trader.memory) % (batch_size / 2) == 0:
            trader.batch_train(batch_size)

        if episode % 5 == 0:
            trader.save("ai_trader_{}.h5".format(episode))

  0%|          | 0/2471 [00:00<?, ?it/s]

Episode: 1/100
AI Trader bought:  $ 5.155800
AI Trader sold:  $ 5.155400  Profit: - $ 0.000400
AI Trader bought:  $ 5.155500
AI Trader sold:  $ 5.153400  Profit: - $ 0.002100
AI Trader bought:  $ 5.154500
AI Trader bought:  $ 5.155000
AI Trader bought:  $ 5.157500
AI Trader sold:  $ 5.157000  Profit: $ 0.002500
AI Trader bought:  $ 5.157500
AI Trader bought:  $ 5.158600
AI Trader sold:  $ 5.160500  Profit: $ 0.005500
AI Trader bought:  $ 5.157800
AI Trader bought:  $ 5.160500
AI Trader bought:  $ 5.160000
AI Trader bought:  $ 5.160500
AI Trader bought:  $ 5.162500
AI Trader bought:  $ 5.163200
AI Trader bought:  $ 5.162500
AI Trader sold:  $ 5.163500  Profit: $ 0.006000
AI Trader sold:  $ 5.163000  Profit: $ 0.005500
AI Trader bought:  $ 5.164700
AI Trader sold:  $ 5.163900  Profit: $ 0.005300
AI Trader bought:  $ 5.163800
AI Trader bought:  $ 5.165000
AI Trader bought:  $ 5.166200
AI Trader sold:  $ 5.168500  Profit: $ 0.010700
AI Trader bought:  $ 5.165000


  2%|▏         | 48/2471 [00:01<01:19, 30.36it/s]

AI Trader sold:  $ 5.162700  Profit: $ 0.002200
AI Trader bought:  $ 5.163500
AI Trader sold:  $ 5.163200  Profit: $ 0.003200
AI Trader sold:  $ 5.163100  Profit: $ 0.002600
AI Trader bought:  $ 5.164000
AI Trader sold:  $ 5.165100  Profit: $ 0.002600
AI Trader sold:  $ 5.162500  Profit: - $ 0.000700
AI Trader bought:  $ 5.167300
AI Trader bought:  $ 5.165300
AI Trader sold:  $ 5.166500  Profit: $ 0.004000
AI Trader sold:  $ 5.165900  Profit: $ 0.001200
AI Trader bought:  $ 5.164900


  3%|▎         | 64/2471 [00:02<01:42, 23.43it/s]

...

AI Trader bought:  $ 5.151600


 11%|█         | 266/2471 [04:25<34:27,  1.07it/s]

AI Trader sold:  $ 5.147500  Profit: - $ 0.004100


 19%|█▉        | 477/2471 [08:17<37:00,  1.11s/it]

KeyboardInterrupt: 